In [ ]:
%qtconsole

In [1]:
# the required python libraries imported
import bnpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.core.validation import silence
from bokeh.core.validation.warnings import MISSING_RENDERERS
from bokeh.layouts import column
from IPython.core.display import display, HTML
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# indicates to jupyer how the plots are to be displayed and sized and some other
# housekeeping particular to this notebook
display(HTML("<style>div.output_scroll { height: 600em; }</style>"))
silence(MISSING_RENDERERS, True)
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 3]

# Theses values need to make sense, mod(data_init_size, batch_size) == 0
init_start = 0
data_start = 20000
data_end = 400000
batch_size = 2000
window_size_in_batches = 5

# all_data contains all the data
all_data = pd.read_csv('../data/anomaly0245.csv')
all_data.drop(all_data.columns[0], inplace=True, axis=1)

# simulation_data is the set used in the simulation 
# not put into a an xdata object, because xdata subsets
# contain metadata about the entire data set, so just testing 
# if that has any side-effects for the algorithm
simulation_data = all_data[data_start:data_end]

# init_data contains the initialization data
init_data = all_data[init_start:window_size_in_batches * batch_size]
init_data = bnpy.data.XData.from_dataframe(init_data)

windows = []
i = 0
while i * batch_size < (len(simulation_data) - window_size_in_batches * batch_size):
    df = simulation_data[i * batch_size:i * batch_size + window_size_in_batches * batch_size]
    windows.append(bnpy.data.XData.from_dataframe(df))
    i += 1

# Graph the data for inspection if required
# p = figure(title="Streaming Data", x_axis_label='x', y_axis_label='y', plot_height=350, plot_width=1200)
# add a line renderer with legend and line thickness
# p.line(all_data.index.tolist(), all_data['anomaly'].tolist(), legend_label="Temp.", line_width=2)
# show the results
# show(p)

In [3]:
gamma = 1.0
sF = 1.0
K = 25  # Initialize K component - this value places a max K the model can develop
nLap = 100

In [4]:
# vars(warm_start_model.obsModel.Post)
# vars(warm_start_model.obsModel.Prior)

In [5]:
ph = 200
pw = 800
bokeh.io.reset_output()
bokeh.io.output_notebook()
output_notebook()                
p1 = figure(title="Dataset", plot_height=ph, plot_width=pw)
p2 = figure(title="Sufficient Statistics", plot_height=ph, plot_width=pw, x_range=p1.x_range)
p3 = figure(title="Average K Resp", plot_height=ph, plot_width=pw, x_range=p1.x_range)
#p6 = figure(title="K", plot_height=ph, plot_width=pw, x_range=p1.x_range)
p = column(p1, p2, p3)#, p6)
target = show(p, notebook_handle=True)

Loading BokehJS ...

Loading BokehJS ...

In [6]:
iname='randexamplesbydist'
opath = f'/tmp/AsteriskK8/coldstart-K=10/b0'  # Dynamic output path according to batch
for ii, window in enumerate(windows):
    start = time.time()

    # Put together the warm start model - starting from the previous iteration 
    output_path = f'/tmp/AsteriskK8/warmstart-K=10/b{ii}'  # Dynamic output path according to batch
    warm_start_model, warm_info_dict = bnpy.run(
        window, 'DPMixtureModel', 'DiagGauss', 'memoVB',
        output_path=output_path,
        nLap=nLap, nTask=1, nBatch=window_size_in_batches, convergeThr=0.0001,
        gamma0=gamma, sF=sF, ECovMat='eye',
        K=K, 
        moves='birth,merge,delete,shuffle',
        initname=iname,
        ts=True, debug=False, verbose=0)
    iname=warm_info_dict['task_output_path']
    opath = f'/tmp/AsteriskK8/warmstart-K=10/b{ii +  1}'

    LP = warm_start_model.calc_local_params(window)
    SS = warm_start_model.get_global_suff_stats(window, LP)
    K_resp = np.mean(LP["resp"], axis=0)
    x_window  = SS.x
    xx_window = SS.xx
    x_window = np.vstack(x_window)
    xx_window = np.vstack(xx_window)
    
    index = int(ii * batch_size) + data_start + window_size_in_batches * batch_size
    x_window = np.squeeze(np.squeeze(x_window)).reshape((1,))
    xx_window = np.squeeze(np.squeeze(xx_window)).reshape((1,))

    y = np.squeeze(window.X[-2000:])#[0::100]
    x = list(range(ii * batch_size + data_start, ii * batch_size + data_start + window_size_in_batches * batch_size))[-2000:]#[0::100]
    end = time.time()
    elapsed = end - start

    # Setup the bokeh plots and render via call back
    print(type(warm_start_model.obsModel.Post.K),warm_start_model.obsModel.Post.K)
    line1 = p1.line(x = x, y = y, color='blue', name='g1', line_width=1)
    line2 = p2.scatter(x = np.squeeze(warm_start_model.obsModel.Post.K).reshape((1,)) * [index], y = x_window, color='blue', name='g1', line_width=1)
    line4 = p3.scatter(x = np.squeeze(warm_start_model.obsModel.Post.K).reshape((1,)) * [index], y = K_resp , color='red', name='g4')
#     line7 = p6.scatter(x = index, y = warm_start_model.obsModel.Post.K , color='red', name='g4')
    push_notebook(handle = target)

  --ts
Dataset Summary:
X Data
  total size: 10000 units
  batch size: 2000 units
  num. batches: 5
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 3 
  beta = [ 1] 
  Expectations
  E[  mean[k]] = 
  [ 0]
  E[ covar[k]] = 
  [[1.]]
Initialization:
  initname = randexamplesbydist
  K = 25 (number of clusters)
  seed = 1607680
  elapsed_time: 0.0 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /tmp/AsteriskK8/warmstart-K=10/b0/1
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
DELETE @ lap 1.00: Disabled. Cannot delete before first complete lap, because SS that represents whole dataset is required.
    0.200/100 after      0 sec. |    290.3 MiB | K   25 | loss -5.659011150e-01 |  
    0.400/100 after      1 sec. |  

   23.000/100 after      9 sec. |    290.9 MiB | K    1 | loss -1.470008378e+00 | Ndiff    0.000 
MERGE @ lap 24.00: No promising candidates, so no attempts.
DELETE @ lap 24.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 24.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
   24.000/100 after      9 sec. |    290.9 MiB | K    1 | loss -1.470008378e+00 | Ndiff    0.000 
... done. converged.
<class 'int'> 1
  --ts
Dataset Summary:
X Data
  total size: 10000 units
  batch size: 2000 units
  num. batches: 5
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 3 
  beta = [ 1] 
  Expectations
  E[  mean[k]] = 
  [ 0]
  E[ covar[k]] = 
  [[1.]]
Initialization:
  initname = /tmp/AsteriskK8/warmstart-K=10/b0/1
  K = 25 (number of clusters)
  seed = 1607680
  elapsed_time: 0.0 sec

DELETE @ lap 5.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 5.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    5.000/100 after      1 sec. |    292.0 MiB | K    1 | loss -1.463536616e+00 | Ndiff    0.000 
MERGE @ lap 6.00: No promising candidates, so no attempts.
DELETE @ lap 6.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 6.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    6.000/100 after      1 sec. |    292.0 MiB | K    1 | loss -1.463536616e+00 | Ndiff    0.000 
MERGE @ lap 7.00: No promising candidates, so no attempts.
DELETE @ lap 7.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 7.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    7.000/100 after      1 sec. |    292.0 MiB | K    1 | loss -1.463536616e+00 | Ndiff    0.000 
MERGE @ lap 8.00: No promising candidates, so no attempts.
DELETE @ lap 8.00: Inelig

... done. converged.
<class 'int'> 1
  --ts
Dataset Summary:
X Data
  total size: 10000 units
  batch size: 2000 units
  num. batches: 5
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 3 
  beta = [ 1] 
  Expectations
  E[  mean[k]] = 
  [ 0]
  E[ covar[k]] = 
  [[1.]]
Initialization:
  initname = /tmp/AsteriskK8/warmstart-K=10/b3/1
  K = 25 (number of clusters)
  seed = 1607680
  elapsed_time: 0.0 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /tmp/AsteriskK8/warmstart-K=10/b4/1
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
DELETE @ lap 1.00: Disabled. Cannot delete before first complete lap, because SS that represents whole dataset is required.
    0.200/100 after      0 sec. |    292.3 MiB | K    1 | loss -1

BIRTH @ lap 6.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    6.000/100 after      1 sec. |    292.6 MiB | K    1 | loss -1.478283616e+00 | Ndiff    0.000 
MERGE @ lap 7.00: No promising candidates, so no attempts.
DELETE @ lap 7.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 7.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    7.000/100 after      1 sec. |    292.6 MiB | K    1 | loss -1.478283616e+00 | Ndiff    0.000 
MERGE @ lap 8.00: No promising candidates, so no attempts.
DELETE @ lap 8.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 8.00 : Added 0 states. 0/1 succeeded. 0/1 failed eval phase. 1/1 failed build phase.
    8.000/100 after      2 sec. |    292.6 MiB | K    1 | loss -1.478283616e+00 | Ndiff    0.000 
MERGE @ lap 9.00: No promising candidates, so no attempts.
DELETE @ lap 9.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 9.00 : Added 

...

DELETE @ lap 22.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 22.00 : Added 3 states. 1/1 succeeded. 0/1 failed eval phase. 0/1 failed build phase.
   22.000/100 after      8 sec. |    315.6 MiB | K    4 | loss -1.032596024e+00 | Ndiff    0.000 
WARNING @ 22.200: loss increased!
    prev = -1.032596023812700e+00
     cur = -6.926773690487791e-01

WARNING @ 22.400: loss increased!
    prev = -6.926773690487791e-01
     cur = -4.995015171516654e-01

WARNING @ 22.600: loss increased!
    prev = -4.995015171516654e-01
     cur = -4.868848808568069e-01

BIRTH @ lap 23.000 : None attempted. 0 past failures. 0 too small. 4 too busy.
DELETE @ lap 23.00: 1/1 accepted. Ndiff 3621.07.
   23.000/100 after      8 sec. |    315.6 MiB | K    3 | loss -1.150503052e+00 | Ndiff 1242.636 
WARNING @ 23.200: loss increased!
    prev = -1.150503052248450e+00
     cur = -7.749862441694231e-01

WARNING @ 23.400: loss increased!
    prev = -7.749862441694231e-01
     cur = -5.993457686428

WARNING @ 42.200: loss increased!
    prev = -1.150503085066898e+00
     cur = -7.749863494591280e-01

WARNING @ 42.400: loss increased!
    prev = -7.749863494591280e-01
     cur = -5.993459434456156e-01

WARNING @ 42.600: loss increased!
    prev = -5.993459434456156e-01
     cur = -5.905986054241356e-01

BIRTH @ lap 43.000 : None attempted. 0 past failures. 0 too small. 3 too busy.
DELETE @ lap 43.00: 1/1 accepted. Ndiff 5178.22.
   43.000/100 after     14 sec. |    315.6 MiB | K    2 | loss -7.117934536e-01 | Ndiff  882.130 
BIRTH @ lap 44.000 : None attempted. 0 past failures. 0 too small. 2 too busy.
DELETE @ lap 44.00: 1/1 accepted. Ndiff 10000.00.
   44.000/100 after     14 sec. |    315.6 MiB | K    1 | loss -7.117934537e-01 | Ndiff    0.000 
MERGE @ lap 45.00: No promising candidates, so no attempts.
DELETE @ lap 45.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 45.00 : Added 2 states. 1/1 succeeded. 0/1 failed eval phase. 0/1 failed build phase.
   45.00

DELETE @ lap 64.00: 1/1 accepted. Ndiff 5214.63.
   64.000/100 after     20 sec. |    315.6 MiB | K    2 | loss -7.117934536e-01 | Ndiff  933.533 
BIRTH @ lap 65.000 : None attempted. 0 past failures. 0 too small. 2 too busy.
DELETE @ lap 65.00: 1/1 accepted. Ndiff 10000.00.
   65.000/100 after     20 sec. |    315.6 MiB | K    1 | loss -7.117934537e-01 | Ndiff    0.000 
MERGE @ lap 66.00: No promising candidates, so no attempts.
DELETE @ lap 66.00: Ineligible. Did not find >= 2 UIDs in entire model.
BIRTH @ lap 66.00 : Added 3 states. 1/1 succeeded. 0/1 failed eval phase. 0/1 failed build phase.
   66.000/100 after     21 sec. |    315.6 MiB | K    4 | loss -1.048266861e+00 | Ndiff    0.000 
WARNING @ 66.200: loss increased!
    prev = -1.048266861479372e+00
     cur = -6.926050653154128e-01

WARNING @ 66.400: loss increased!
    prev = -6.926050653154128e-01
     cur = -4.995016471315568e-01

WARNING @ 66.600: loss increased!
    prev = -4.995016471315568e-01
     cur = -4.8688454873

WARNING @ 85.600: loss increased!
    prev = -5.980155644022122e-01
     cur = -5.790678096933145e-01

BIRTH @ lap 86.000 : None attempted. 0 past failures. 0 too small. 4 too busy.
DELETE @ lap 86.00: 1/1 accepted. Ndiff 3540.99.
   86.000/100 after     26 sec. |    315.6 MiB | K    3 | loss -1.150502419e+00 | Ndiff  971.646 
WARNING @ 86.200: loss increased!
    prev = -1.150502418652061e+00
     cur = -7.749842125625718e-01

WARNING @ 86.400: loss increased!
    prev = -7.749842125625718e-01
     cur = -5.993423907340149e-01

WARNING @ 86.600: loss increased!
    prev = -5.993423907340149e-01
     cur = -5.905956671245284e-01

BIRTH @ lap 87.000 : None attempted. 0 past failures. 0 too small. 3 too busy.
DELETE @ lap 87.00: 1/1 accepted. Ndiff 5178.22.
   87.000/100 after     26 sec. |    315.6 MiB | K    2 | loss -7.117934536e-01 | Ndiff  882.137 
BIRTH @ lap 88.000 : None attempted. 0 past failures. 0 too small. 2 too busy.
DELETE @ lap 88.00: 1/1 accepted. Ndiff 10000.00.
   88.0

ValueError: cannot reshape array of size 2 into shape (1,)